In [3]:
%pip install requests
import requests
import zipfile
import io
import csv

# URL of the ZIP file containing the CSV
csv_url = 'https://cdn.sanity.io/files/jo7n4k8s/production/262f04c41d99fea692e0125c342e446782233fe4.zip/stack-overflow-developer-survey-2024.zip'

# Download the ZIP file content
response = requests.get(csv_url)
if response.status_code == 200:
    zip_content = io.BytesIO(response.content)

    # Extract the ZIP file
    with zipfile.ZipFile(zip_content, 'r') as zip_ref:
        # Iterate through the files in the ZIP
        for file_name in zip_ref.namelist():
            # Check if the file is a CSV
            if file_name.endswith('.csv'):
                with zip_ref.open(file_name) as csv_file:
                    # Parse CSV content with newline='' to handle new-line characters properly
                    reader = csv.reader(io.TextIOWrapper(csv_file, newline='', encoding='utf-8'), skipinitialspace=True)
                    for row in reader:
                        pass
                        #print(row)
            else:
                pass
                #print(f"Skipping non-CSV file: {file_name}")
else:
    print("Failed to retrieve the ZIP file.")

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 64 kB 7.3 MB/s eta 0:00:011
     |████████████████████████████████| 70 kB 8.7 MB/s  eta 0:00:01
     |████████████████████████████████| 197 kB 18.2 MB/s eta 0:00:01
     |████████████████████████████████| 166 kB 14.9 MB/s eta 0:00:01
     |████████████████████████████████| 128 kB 13.8 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


/Users/adriannica1020/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
